# 1. Import the necessary libraries for you to be able to query 4square API

In [3]:
import requests
import json
import pandas as pd
from getpass import getpass
import os
import time
from pymongo import MongoClient
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
token = os.getenv("token")

In [15]:
def requests_for_foursquare (query, lat, lon, radius=None, limit=1, sort='DISTANCE'):

    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&sort={sort}&limit={limit}"

    if radius:
        url += f'&radius={radius}'

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

# 2. Get names and location of the 10 coffee places by passing Ironhack as your location

In [16]:
where = "Pamplona, 96, 08018 Barcelona"
url_geocode = f"https://geocode.xyz/{where}?json=1"
response = requests.get(url_geocode)
response = response.json()

res = requests_for_foursquare("coffee", response['latt'], response['longt'], limit=10)
res = res['results']
res[0]

{'fsq_id': '556d6217498ef50c5d323048',
 'categories': [{'id': 13003,
   'name': 'Bar',
   'short_name': 'Bar',
   'plural_name': 'Bars',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
    'suffix': '.png'}},
  {'id': 13035,
   'name': 'Coffee Shop',
   'short_name': 'Coffee Shop',
   'plural_name': 'Coffee Shops',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
    'suffix': '.png'}}],
 'chains': [],
 'closed_bucket': 'VeryLikelyOpen',
 'distance': 450,
 'geocodes': {'main': {'latitude': 41.397977, 'longitude': 2.195589},
  'roof': {'latitude': 41.397977, 'longitude': 2.195589}},
 'link': '/v3/places/556d6217498ef50c5d323048',
 'location': {'address': 'Calle de Pujades, 95',
  'admin_region': 'Cataluña',
  'country': 'ES',
  'cross_street': 'C. de Badajoz',
  'formatted_address': 'Calle de Pujades, 95 (C. de Badajoz), 08005 Barcelona',
  'locality': 'Barcelona',
  'postcode': '08005'},
 'name': "Nømad Roaster's Home",
 'r

In [12]:
def get_name_location(rest):
    return {'name': rest['name'], 'location': rest['location'], 'latitude': rest['geocodes']['main']['latitude'], 'longitude': rest['geocodes']['main']['longitude']}

coffes = [get_name_location(i) for i in res]
coffes[0:2]

[{'name': "Nømad Roaster's Home",
  'location': {'address': 'Calle de Pujades, 95',
   'admin_region': 'Cataluña',
   'country': 'ES',
   'cross_street': 'C. de Badajoz',
   'formatted_address': 'Calle de Pujades, 95 (C. de Badajoz), 08005 Barcelona',
   'locality': 'Barcelona',
   'postcode': '08005'},
  'latitude': 41.397977,
  'longitude': 2.195589},
 {'name': 'Three Marks Coffee',
  'location': {'address': 'Ausiàs Marc, 151',
   'admin_region': 'Cataluña',
   'country': 'ES',
   'formatted_address': 'Ausiàs Marc, 151, 08013 Barcelona Catalunya',
   'locality': 'Barcelona',
   'postcode': '08013',
   'region': 'Catalunya'},
  'latitude': 41.397168,
  'longitude': 2.183123}]

# 3. Now get the 5 CLOSEST coffe places. Make use of params: limit, near?

In [17]:
res = requests_for_foursquare("coffee", response['latt'], response['longt'], limit=5)
res = res['results']
coffes = [get_name_location(i) for i in res]
coffes

[{'name': "Nømad Roaster's Home",
  'location': {'address': 'Calle de Pujades, 95',
   'admin_region': 'Cataluña',
   'country': 'ES',
   'cross_street': 'C. de Badajoz',
   'formatted_address': 'Calle de Pujades, 95 (C. de Badajoz), 08005 Barcelona',
   'locality': 'Barcelona',
   'postcode': '08005'},
  'latitude': 41.397977,
  'longitude': 2.195589},
 {'name': 'Three Marks Coffee',
  'location': {'address': 'Ausiàs Marc, 151',
   'admin_region': 'Cataluña',
   'country': 'ES',
   'formatted_address': 'Ausiàs Marc, 151, 08013 Barcelona Catalunya',
   'locality': 'Barcelona',
   'postcode': '08013',
   'region': 'Catalunya'},
  'latitude': 41.397168,
  'longitude': 2.183123},
 {'name': 'Brew Coffee',
  'location': {'address': 'Calle de Roger de Flor, 102',
   'admin_region': 'Cataluña',
   'country': 'ES',
   'cross_street': '',
   'formatted_address': 'Calle de Roger de Flor, 102, 08013 Barcelona Catalunya',
   'locality': 'Barcelona',
   'postcode': '08013',
   'region': 'Catalunya'

# 4. Define a function that returns name, lat, long for starbucks OR restaurants AND for any given location

In [27]:
def foursquare_places(venue, coordinates):
    lat, lon = str(coordinates).split(',')
    res = requests_for_foursquare (venue, lat, lon, radius = 500, sort = "DISTANCE", limit = 10)
    
    try:
        data = res['results']
        results = []
        
        for i in data:
            name = i['name']
            location = i['geocodes']['main']
            lat = location.get('latitude')
            lng = location.get('longitude')
            results.append({
                'name': name,
                'lat': lat,
                'lng': lng
            })
        return results
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return []

In [28]:
res = foursquare_places("coffee", response['latt'] + ',' + response['longt'])
res

[{'name': 'Espacio 88', 'lat': 41.396993, 'lng': 2.191171},
 {'name': 'SKYE Coffee Co', 'lat': 41.396937, 'lng': 2.191519},
 {'name': 'Syra Coffee Poblenou', 'lat': 41.396518, 'lng': 2.19417},
 {'name': 'Espai Joliu', 'lat': 41.398735, 'lng': 2.195069},
 {'name': 'Granja Mabel', 'lat': 41.394816, 'lng': 2.187354},
 {'name': 'Eva-2', 'lat': 41.401636, 'lng': 2.189006},
 {'name': 'Cafeteria Industrial', 'lat': 41.399175, 'lng': 2.195368},
 {'name': 'Market Cuina Fresca', 'lat': 41.398365, 'lng': 2.195559},
 {'name': "Nømad Roaster's Home", 'lat': 41.397977, 'lng': 2.195589},
 {'name': 'Mare Meva', 'lat': 41.402167, 'lng': 2.189879}]

# 5. Export it into data folder as a json: `data/coffe_shops.json`

In [29]:
pd.DataFrame(res).to_json('data/coffe_shops.json')

# BONUS: load it into MongoDB in a collection named "ironcoffee" using pymongo

In [31]:
client = MongoClient("localhost:27017")
db = client["ironhack"]
collection = db['ironcoffee']
for coffe in coffes:
    collection.insert_one(coffe)